In [97]:
from Bio import SeqIO
from Bio import Seq
from antibody.anarci import Anarci
anarci = Anarci()

# Cats and dogs from segment libs

'IMGT000003|IGLJ9*01|Canis lupus familiaris_boxer|F|J-REGION|2580610..2580647|38 nt|2| | || |38+0=38| | |'

In [171]:
import pandas as pd

v_genes = pd.read_csv(
    "/Users/jwillis/repos/bfx-bp/bfx-bp-antibody/antibody/data/VSEGMENT.csv.gz",
    index_col=0,
)
j_gene = pd.read_csv(
    "/Users/jwillis/repos/bfx-bp/bfx-bp-antibody/antibody/data/JSEGMENT.csv.gz",
    index_col=0,
)
imgt_df = pd.read_csv(
    "/Users/jwillis/repos/bfx-bp/bfx-bp-reference/bfx_bp_reference/data/IMGT_REFERENCE.csv.gz",
    index_col=0,
)

lookup_dictionary = {
    **imgt_df[["latin", "common"]].drop_duplicates().set_index("common").to_dict(),
    **imgt_df[["latin", "common"]].drop_duplicates().set_index("latin").to_dict(),
}

<IPython.core.display.Javascript object>

In [329]:
latin_lookup = {"dog": "Canis_lupus", "cat": "Felus_catus"}
for species in ["cat", "dog"]:
    for chain in ["H", "K", "L"]:
        lookup = latin_lookup[species]
        with open(
            f"build_pipeline/IMGT_sequence_files/fastafiles/{lookup}_{chain}V.fasta",
            "w",
        ) as f:
            sub_df = imgt_df[imgt_df["common"] == species]
            sub_df = sub_df[sub_df["gene"].str[2] == chain]
            # "IMGT000003|IGLJ9*01|Canis lupus familiaris_boxer|F|J-REGION|2580610..2580647|38 nt|2| | || |38+0=38| | |"
            for entry in sub_df.iterrows():
                index = entry[0]
                series = entry[1]
                gene = series["gene"]
                functional = series["functional"]
                if len(gene.split("*")) == 1:
                    gene += "*01"
                latin = series["latin"]
                series = series.fillna("")
                vdj_nt = "".join(series[["FW1", "CDR1", "FW2", "CDR2", "FW3", "CDR3"]])
                vdj_aa = str(Seq.Seq(vdj_nt).translate())
                if vdj_aa.count("*") > 3:
                    continue
                header = f"{index}|{gene}|{latin}|{functional}|V-REGION||{len(vdj_aa)} AA|||||||||"
                try:
                    result = anarci.run_single(gene, vdj_aa)
                except KeyError:
                    print(f"having trouble with {header}-{vdj_aa}")
                vdj = result.segment_table["vdj"]
                if result.segment_table["tail"]:
                    vdj += result.segment_table["tail"]
                vdj_mod = vdj[:-20] + vdj[-20:].replace("-", "")
                vdj_mod = vdj_mod.replace("-", ".")
                f.write(f">>{header}\n{vdj_mod}\n")
                print(f"{header}\n{vdj_mod}\n{vdj_nt}")

/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "
/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "
/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a m

4|IGHV1-1*01|Felis_catus_Abyssinian|F|V-REGION||96 AA|||||||||
DVQLVESGG.DLVQPGGSLRLTCVASGFTF....SSYEMNWVRQAPGKGLQWVAYISSG...GSTYYADSVK.GRFTISRDNAKNTLYLQMNSLKTEDTATYYCA
GACGTGCAGCTGGTGGAGTCTGGGGGAGACCTGGTGCAGCCTGGGGGGTCCCTGAGACTCACCTGTGTGGCCTCTGGATTCACCTTCAGTAGCTATGAAATGAACTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGCAGTGGGTCGCATATATTAGTAGTGGAGGTAGCACATACTACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACACGCTGTATCTGCAGATGAACAGCCTCAAGACCGAGGACACGGCCACATATTACTGTGCGA
35|IGHV1-2*01|Felis_catus_Abyssinian|F|V-REGION||97 AA|||||||||
QVLLVQSGA.EVRKPGASVKIFCKVSGYTF....TDYYIHWLRQAPGQGLEWMGRIDPE..DGSTTYAQKFQ.GRLTLTADTSTNTAYMELSSLRSADTAMYYCA
CAGGTTTTGCTGGTGCAGTCTGGGGCTGAAGTGAGGAAGCCTGGGGCATCAGTGAAGATCTTCTGCAAAGTATCTGGATACACCTTCACTGATTACTATATACACTGGTTGCGACAGGCTCCTGGACAAGGGCTTGAGTGGATGGGAAGAATTGACCCTGAAGATGGTTCTACAACCTATGCACAGAAGTTCCAGGGCAGACTCACCCTGACAGCAGACACATCCACAAACACAGCCTACATGGAGTTGAGCAGCTTGAGGTCTGCAGACACAGCCATGTATTACTGTGCAA
246|IGHV10-1*01|Felis_catus_Abyssinian|F|V-REGION||95 AA|||||||||
QLTLRESGP.GL

/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "
/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "
/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a m

274|IGHV11-1*01|Felis_catus_Abyssinian|F|V-REGION||97 AA|||||||||
DVQLVESGG.DLVKPGGSLRLTCVASGFSF....SSYAMSWVRQAPGKGLQWVASISYD..GGSIGYADSVK.GRFTISRDNAQNTLYLQMNSLKTEDTATYYCA
GACGTGCAGCTGGTGGAGTCTGGGGGAGACCTGGTGAAGCCCGGGGGGTCCCTGAGACTCACCTGTGTGGCCTCTGGATTCAGCTTCAGTAGTTATGCAATGAGCTGGGTCCGCCAGGCTCCAGGAAAGGGGCTGCAGTGGGTCGCATCAATTAGTTATGATGGAGGTAGTATAGGCTACGCAGACTCCGTGAAGGGCCGATTTACCATCTCCAGAGACAACGCCCAGAACACACTGTATCTGCAGATGAACAGCCTCAAGACCGAGGACACGGCCACATATTACTGTGCGA
281|IGHV11-2*01|Felis_catus_Abyssinian|F|V-REGION||96 AA|||||||||
DVQLVESGG.DLVKPGGSLRLTCVASGFNV....DDYALSWVRQAPGKGLQWVSTIVTA...GTTYYTDSVK.GRFTISRDNAKNTLYLQMNSLTTEDTATYYCA
GATGTGCAGTTGGTGGAGTCTGGGGGAGACCTGGTGAAGCCTGGGGGGTCCCTGAGACTCACCTGTGTAGCCTCTGGATTCAACGTCGATGACTATGCATTGAGTTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGCAGTGGGTCTCAACAATTGTGACCGCAGGTACCACGTATTACACAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACACGCTATATCTGCAGATGAACAGCCTGACGACTGAGGACACGGCCACATATTACTGTGCGA
291|IGHV12-1*01|Felis_catus_Abyssinian|F|V-REGION||97 AA|||||||||
DVQLVES

/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "
/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "
/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a m

...

2193|IGLV8-88-4*01|Canis_lupus_familiaris_boxer|P|V-REGION||102 AA|||||||||
QTVVTQDPS.LSVSPGGTVTLTCGLSSGSVS...TSNYPGWYQQTQVKAPCMLIYST.......NSYPSGVP.NCFTGSI..SGKKAALTITGDQPEDET........IIVHCIWVVHL
CAGACTGTGGTGGCTCAGGAGTCATCAGTCTCAGTGTCTCCAGGAGGGACAGTCACACTCACTTGTGGCCTCAGCTCTGGGTCAGTGACTACAAGTAACTACCACAGCTGGTACCAGCGGACCCAAGGCCGGTCTCCTCACATGCTTATCTATGACACAAGCAGCCGTCCTTCTGAGGTCCTGATCGCTTCCCTGGTTCCATCTCTGGGAACAAAGCTGCCCTCACTGTCAGAGGAGCCCAGCCTGAGGACGAGGCTGACTACTACTGTGGCATGCATGATGTCAGTGGGAGGAATTACAATTACC
Did not detect anything in summary dataframe,[],[]
having trouble with 2194|IGLV8-89-3*01|Canis_lupus_familiaris_boxer|P|V-REGION||100 AA|||||||||-QIVVARRHCCQCLQEGESHSLVASALGQSLQVTTPTGSSRPQGGLLARLSTAQKTAPLGSLTASLDPSLGTKPPSPSQEPSLRTRLLLFYTTW*WELLHL
2194|IGLV8-89-3*01|Canis_lupus_familiaris_boxer|P|V-REGION||100 AA|||||||||
QTVVTQDPS.LSVSPGGTVTLTCGLSSGSVS...TSNYPGWYQQTQVKAPCMLIYST.......NSYPSGVP.NCFTGSI..SGKKAALTITGDQPEDET........IIVHCIWVVHL
CAGATTGTGGTGGCCAGGAGGCATTGTTGTCAGTGTCTCCAGGAGGGAGAGTC

/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "
/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "
/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a m

2198|IGLV8-90-4*01|Canis_lupus_familiaris_boxer|P|V-REGION||103 AA|||||||||
QTVVTQEPS.LSVSPGGTVTLTCGLSSGSVS...TGNKPGWYQHTPGQAPRRIIYDT.......SSRPSGVP.DRFSGSI..SENKTALTITEAQPEDEADYIIYEWWCL
CATATTTTGGTGACTCAGGAGCCATCACTGTCAGTGTCTCCATGAGGGACAGTCACACTCACTTGTGGCCTCAGCTCTGGGTCAGTCACTACAAGTAACTACCCCAGGTATACCAGCAGAACCCAGGCAAGGCTCCTAGCACAGTTATCTACAACAAAAACAGCTGCCCCTCTGGGGTCCATGGTCGATTCTCTGGATCCATCTCTGGAAGCAAAGCCGCCTTCACAATCACAGGAGCCCAGCCTGAGGTTGAGGCTGACTACTACTGTGTTACAGAACATGGCTCCTCACATGGGAACAGCCTCACTCAC
2199|IGLV8-92-1*01|Canis_lupus_familiaris_boxer|P|V-REGION||97 AA|||||||||
QTVVTQDPS.LSVSPGGTVTFTCGLSSG*VS...TRNYPSWYQQTQGQAPCMLIYST.......SRHPSGVP.DRFSGSI..SGNKVALTITGAQPEDKT........IIVHCIWVVHL
CAGACTGTGGTCACCCAGGATCCGTCACTCTCAGTGTCTCCAGGAGGGACAGTCACATTCACATGTGGCCTCAGCTCTGGGTAAGTCTCTACAAGAAACTACCCCAGCTGGTACCAGCAGACCCAAGGCCAGGCTCCTTGCATGCTTATCTACAGCACAAGCAGACACCCTTCTGGGGTCCCTGATCGCTTCTCTGGATCCATCTCTGGGAACAAAGTCGCCCTCACCATCACAGGAGCCCAGCCTGAGGATAAGACTATTATTGTTCACTGCATATGGGTAGTACATTTA
2200|IGLV8-93*

/Users/jwillis/miniconda3/envs/bfx-bp-antibody/lib/python3.8/site-packages/Bio/Seq.py:2573: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn("Partial codon, len(sequence) not a multiple of three. "


<IPython.core.display.Javascript object>

In [321]:
vdj

'DVQLVESGG-DLVQPGGSLRLTCVASGFTF----SSYEMNWVRQAPGKGLQWVAYISSG---GSTYYADSVK-GRFTISRDNAKNTLYLQMNSLKTEDTATY---------------'

<IPython.core.display.Javascript object>

In [305]:
Seq.Seq("".join(series[["FW1", "CDR1", "FW2", "CDR2", "FW3", "CDR3"]])).translate()

Seq('EVQLVQSGAEVKKPGTSVKVSCKTSGYTFTDYYMYWVRQASGAGLDWMGQIGP*...SVR', HasStopCodon(ExtendedIUPACProtein(), '*'))

<IPython.core.display.Javascript object>

In [330]:
latin_lookup = {"dog": "Canis_lupus", "cat": "Felus_catus"}
for species in ["cat", "dog"]:
    for chain in ["H", "K", "L"]:
        lookup = latin_lookup[species]
        with open(
            f"build_pipeline/IMGT_sequence_files/fastafiles/{lookup}_{chain}J.fasta",
            "w",
        ) as f:
            sub_df = j_gene[j_gene["species"] == species]
            sub_df = sub_df[sub_df["full"].str[2] == chain]
            for entry in sub_df.iterrows():
                index = entry[0]
                series = entry[1]
                gene = series["full"]
                functional = "F"
                if len(gene.split("*")) == 1:
                    gene += "*01"
                latin = lookup
                vdj_aa = "".join(series[["cdr3_aa", "fwr4_aa"]])

                header = f"{index}|{gene}|{latin}|{functional}|J-REGION||{len(vdj_aa)} AA|||||||||"
                f.write(f">>{header}\n{vdj_aa}\n")

<IPython.core.display.Javascript object>

In [283]:
imgt_df[imgt_df["common"] == "cat"]

,gene,latin,common,functional,FW1,CDR1,FW2,CDR2,FW3,CDR3,source
4,IGHV1-1,Felis_catus_Abyssinian,cat,F,GACGTGCAGCTGGTGGAGTCTGGGGGAGACCTGGTGCAGCCTGGGG...,GGATTCACCTTCAGTAGCTATGAA,ATGAACTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGCAGTGGGTCG...,ATTAGTAGTGGAGGTAGCACA,TACTACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACA...,GCGA,imgt_db
35,IGHV1-2,Felis_catus_Abyssinian,cat,F,CAGGTTTTGCTGGTGCAGTCTGGGGCTGAAGTGAGGAAGCCTGGGG...,GGATACACCTTCACTGATTACTAT,ATACACTGGTTGCGACAGGCTCCTGGACAAGGGCTTGAGTGGATGG...,ATTGACCCTGAAGATGGTTCTACA,ACCTATGCACAGAAGTTCCAGGGCAGACTCACCCTGACAGCAGACA...,GCAA,imgt_db
246,IGHV10-1,Felis_catus_Abyssinian,cat,F,CAACTTACACTTCGGGAGTCCGGCCCAGGACTGGTGAAGCCTTCAC...,GGAGGCTCTGTTACCAGCAGTTACTAC,TGGAACTGGATCCGCCAGCGCCCTGGGAGAGGGTTGGAGTGGCTGG...,TGGTCAGGTAGCACC,AGCTACAACCCGGCTTTGCAGGGCCGCATCTCCATCGCTGCTGACA...,GGAA,imgt_db
258,IGHV10-2,Felis_catus_Abyssinian,cat,F,GACGTGCATCTGGTGGAGTCTGGGGGAGACCTGGTGAAGCCTGGGG...,GGAATCACCTTCAGGACTTATTGG,ATGAACTGGGTCCGCCAGGCTCCTGGGAAGGGGCTGCAGTGGGTCT...,ATTAGTGGTACTGGAGGTAATACA,GCCTATACGGACTCCGTGAAGGGCCGATTCACCATTTCCAGAGACA...,GCGA,imgt_db
274,IGHV11-1,Felis_catus_Abyssinian,cat,F,GACGTGCAGCTGGTGGAGTCTGGGGGAGACCTGGTGAAGCCCGGGG...,GGATTCAGCTTCAGTAGTTATGCA,ATGAGCTGGGTCCGCCAGGCTCCAGGAAAGGGGCTGCAGTGGGTCG...,ATTAGTTATGATGGAGGTAGTATA,GGCTACGCAGACTCCGTGAAGGGCCGATTTACCATCTCCAGAGACA...,GCGA,imgt_db
...,...,...,...,...,...,...,...,...,...,...,...
4468,TRGV2-3*01,Felis_catus_Abyssinian,cat,F,GCAGCGATCAGCCTGGGGCAGCCCTCTATGGTCGTGGCGCAGGCAG...,ACCAAGGTCAGCTAC,ATCCACTGGTACCACCACCAGGAAGGTGCGGCCCCCAAGAGGCTCC...,CTGGACATGTCCGGAACGTTTGTG,CAGAAAGATGGGGGCCTGAATGAGGACAAAGTCAATGCCAAGAGGG...,GCTGCCTGGGAAGCT,vquest_fasta
4470,TRGV2-4*01,Felis_catus_Abyssinian,cat,F,GCAGCGATCAGTCTGACTCAGCGGGCTGTGGTGGTGGAGCGCGTGG...,GCCTCCGTCAGCTAC,ATCCACTGGTACCTCCACCAGGAGGGGACTGACCCCAAGAGGATCC...,CTGGACATGTCAAGGTTATCTGTG,GAGAGGTATGGGGGCTTGGAAGCGGACAAAATCGATGCCAAGAAAG...,GCTGCCTGGGAAGCT,vquest_fasta
4510,TRGV5-3*01,Felis_catus_Abyssinian,cat,F,GGCATTTTCATAACACAATTCGTACCATCCATTACCAAGAAAAAAG...,ACGGGTGTCTTAAAGAAAAACGTAAAT,ATACACTGGTATCGACAGAGGCCAGACCAGCCTCTAAAACGCATTC...,ATTTCCTCAAATGAAAATGTG,GTCCACGAACAAGGTATTAGTGAGGAAAAATATGAAGCCAGGAAAC...,GCCTGCTGGGATC,vquest_fasta
4519,TRGV6-1*01,Felis_catus_Abyssinian,cat,P,GCAGAGATCAGGCTGGAGCAGCCAGCTGTGGTCGTGGCTCGTGAGG...,ACCAAGGTCAGCTAC,ACCCACTGGTACCACCACCAGGAGGGAAAGGTCCCCCAAAGGCTTC...,CTGGCCATGTTCCATTAGGATGTG,CAGTGGGAATCTGTCGAGAAAGCAGACAATGTCACGGCCACAGAGG...,GCTGTCTGGGAACCG,vquest_fasta


<IPython.core.display.Javascript object>